In [22]:

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torchsummary import summary
from IPython import display
import matplotlib.pyplot as plt

In [23]:


if torch.cuda.is_available():
    print("robin")

# use GPU if available
device = torch.device('mps')

In [24]:
# 创建高斯分布
nPerClass = 1000
imgSize = 91

x = np.linspace(-4, 4, imgSize)
X, Y = np.meshgrid(x, x)

widths = [1.8, 2.4]

images = torch.zeros(2*nPerClass, 1, imgSize, imgSize)
labels = torch.zeros(2*nPerClass)

for i in range(2*nPerClass):

    # 创建高斯分布

    # 随机中心
    ro = 2 * np.random.randn(2)
    # 高斯分布
    G = np.exp(-((X-ro[0])**2 + (Y-ro[1])**2)/(2**widths[i % 2]))

    G = G + np.random.randn(imgSize, imgSize)/5

    images[i, :, :, :] = torch.Tensor(G).view(1, imgSize, imgSize)

labels = labels[:, None]

### 整理数据

In [25]:

# Step 2: use scikitlearn to split the data
train_data, test_data, train_labels, test_labels = train_test_split(
    images, labels, test_size=.1)

# Step 3: convert into PyTorch Datasets
train_data = TensorDataset(train_data, train_labels)
test_data = TensorDataset(test_data, test_labels)

# Step 4: translate into dataloader objects
batchsize = 32
train_loader = DataLoader(
    train_data, batch_size=batchsize, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0])

In [26]:

# create a class for the model
def makeTheNet(fcUnits):

    class gausnet(nn.Module):
        def __init__(self):
            super().__init__()

            # all layers in one go using nn.Sequential
            self.enc = nn.Sequential(
                # output size: (91+2*1-3)/1 + 1 = 91
                nn.Conv2d(1, 6, 3, padding=1),
                nn.ReLU(),                   # note that relu is treated like a "layer"
                nn.AvgPool2d(2, 2),           # output size: 91/2 = 45
                # output size: (45+2*1-3)/1 + 1 = 45
                nn.Conv2d(6, 4, 3, padding=1),
                nn.ReLU(),                   #
                nn.AvgPool2d(2, 2),           # output size: 45/2 = 22
                nn.Flatten(),                # vectorize conv output
                nn.Linear(22*22*4, 2*fcUnits),  # output size: 2*fcUnits
                # NOTE: in the video I forgot to include ReLU between the linear layers
                nn.ReLU(),
                nn.Linear(2*fcUnits, fcUnits),  # output size: fcUnits
                nn.ReLU(),                   #
                nn.Linear(fcUnits, 1),        # output size: 1
            )

        def forward(self, x):
            return self.enc(x)

    # create the model instance
    net = gausnet()

    # loss function
    lossfun = nn.BCEWithLogitsLoss()

    # optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=.001)

    return net, lossfun, optimizer

In [27]:


# a function that trains the model

def function2trainTheModel(fcUnits):

    # number of epochs
    numepochs = 10

    # create a new model
    net, lossfun, optimizer = makeTheNet(fcUnits)

    # send the model to the GPU
    net.to(device)

    # initialize losses
    trainLoss = torch.zeros(numepochs)
    testLoss = torch.zeros(numepochs)
    trainAcc = torch.zeros(numepochs)
    testAcc = torch.zeros(numepochs)

    # loop over epochs
    for epochi in range(numepochs):

        # loop over training data batches
        batchLoss = []
        batchAcc = []
        for X, y in train_loader:

            # push data to GPU
            X = X.to(device)
            y = y.to(device)

            # forward pass and loss
            yHat = net(X)
            loss = lossfun(yHat, y)

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # loss from this batch
            batchLoss.append(loss.item())
            batchAcc.append(torch.mean(((yHat > 0) == y).float()).item())
        # end of batch loop...

        # and get average losses across the batches
        trainLoss[epochi] = np.mean(batchLoss)
        trainAcc[epochi] = 100*np.mean(batchAcc)

        # test accuracy
        X, y = next(iter(test_loader))  # extract X,y from test dataloader

        # push data to GPU
        X = X.to(device)
        y = y.to(device)

        with torch.no_grad():  # deactivates autograd
            yHat = net(X)
            loss = lossfun(yHat, y)

        # compare the following really long line of code to the training accuracy lines
        testLoss[epochi] = loss.item()
        testAcc[epochi] = 100*torch.mean(((yHat > 0) == y).float()).item()

    # end epochs

    # function output
    return trainLoss, testLoss, trainAcc, testAcc, net

In [28]:
# crazy note: this cell took ~15 minutes on CPU!

# specify number of hidden units
numberOfLinearUnits = np.round(np.linspace(5, 500, 20))

# initialize results matrix
results = np.zeros((len(numberOfLinearUnits), 4))

for i, nunits in enumerate(numberOfLinearUnits):
    trainLoss, testLoss, trainAcc, testAcc, net = function2trainTheModel(
        int(nunits))
    results[i, :] = [trainLoss[-1], testLoss[-1], trainAcc[-1], testAcc[-1]]

In [29]:
ig, ax = plt.subplots(1, 2, figsize=(16, 5))

ax[0].plot(numberOfLinearUnits, results[:, :2], 's-')
ax[0].set_xlabel('Number of units in final linear layer')
ax[0].set_ylabel('Loss (MSE)')
ax[0].set_title('Final model loss')
ax[0].legend(['Train', 'Test'])

ax[1].plot(numberOfLinearUnits, results[:, 2:], 's-')
ax[1].set_xlabel('Number of units in final linear layer')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_title('Final model test accuracy')
ax[1].legend(['Train', 'Test'])

plt.show()

NameError: name 'plt' is not defined

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(16, 5))

ax[0].plot(numberOfLinearUnits, results[:, :2], 's-')
ax[0].set_xlabel('Number of units in final linear layer')
ax[0].set_ylabel('Loss (MSE)')
ax[0].set_title('Final model loss')
ax[0].legend(['Train', 'Test'])

ax[1].plot(numberOfLinearUnits, results[:, 2:], 's-')
ax[1].set_xlabel('Number of units in final linear layer')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_title('Final model test accuracy')
ax[1].legend(['Train', 'Test'])

plt.show()